In [1]:
import matplotlib as plt
print('matplotlib: {}'.format(plt.__version__))
# pandas
import pandas as pd
print('pandas: {}'.format(pd.__version__))
# scikit-learn
import sklearn as sk
print('sklearn: {}'.format(sk.__version__))
from pandas_profiling import ProfileReport
import seaborn as sns
print('seaborn: {}'.format(sns.__version__))
import numpy as np
print('numpy: {}'.format(np.__version__))
from sklearn import linear_model
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

matplotlib: 3.3.2
pandas: 1.1.3
sklearn: 0.23.2
seaborn: 0.11.0
numpy: 1.19.2


In [2]:
X_train = pd.read_csv("X_train.csv")

In [3]:
X_val=pd.read_csv("X_val.csv")
X_test = pd.read_csv("X_test.csv")

In [4]:
X_test.shape

(1459, 78)

In [5]:
y_train = pd.read_csv("y_train.csv")
y_val = pd.read_csv("y_val.csv")

In [86]:
id_test = pd.read_csv("id_test.csv")

In [102]:
sample_submission = pd.read_csv("sample_submission.csv")

In [105]:
id_test2 = sample_submission.Id

In [8]:
X_train.shape

(1019, 79)

In [9]:
X_train = X_train.drop(columns=["Unnamed: 0"])

In [10]:
X_val = X_val.drop(columns=["Unnamed: 0"])

In [11]:
X_test = X_test.drop(columns=["Unnamed: 0"])

In [12]:
y_train =y_train.drop(columns=["Unnamed: 0"])

In [13]:
y_val = y_val.drop(columns=["Unnamed: 0"])

In [14]:
id_test = id_test.drop(columns=["Unnamed: 0"])

In [90]:
id_test.reset_index(inplace=True)

In [93]:
id_test = id_test.drop(columns=["index"])

In [15]:
models = []
models.append(('LR', LinearRegression()))
models.append(('Ridge', linear_model.Ridge()))
models.append(('SVR', svm.SVR()))
models.append(('Lasso',linear_model.Lasso()))
models.append(('ElasticNet',ElasticNet()))
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold,scoring='neg_root_mean_squared_error')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LR: -0.135828 (0.008469)
Ridge: -0.134779 (0.007722)


C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

SVR: -0.336435 (0.017211)
Lasso: -0.302402 (0.020140)
ElasticNet: -0.299317 (0.020669)


In [19]:
# The best model is the ridge model

In [20]:
# Let's make a baseline

In [16]:
model = linear_model.Ridge()
model.fit(X_train, y_train)
predictions = model.predict(X_val)

In [17]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_val, predictions,squared=False)

0.14740600048182884

In [38]:
# Let's tune the hyperparameters

In [20]:
from sklearn.model_selection import GridSearchCV
alphas = np.array([1,0.1,0.01,0.001,0.0001,0])
model = linear_model.Ridge()
grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas), scoring='neg_root_mean_squared_error')
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.best_estimator_.alpha)


-0.13415896351556014
1.0


In [ ]:
# During EDA i took the log of the sale price to have a normally distributed feature
# Now I have to undo it to submit my results.

In [36]:
X_train.shape

(1019, 78)

In [21]:
X_train2 = X_train.drop(columns="Condition2_RRAe")

In [22]:
model2 = linear_model.Ridge()
model2.fit(X_train2, y_train)

Ridge()

In [23]:
X_test.shape

(1459, 77)

In [24]:
#model.fit(X_train, y_train)
predictions2 = model2.predict(X_test)


In [25]:
predictions2.shape

(1459, 1)

In [45]:
SalePrice =  np.exp(predictions2)

In [46]:
SalePrice = pd.DataFrame(SalePrice.flatten())

In [95]:
id_test

,Id
0,1461
1,1462
2,1463
3,1464
4,1465
...,...
1454,2915
1455,2916
1456,2917
1457,2918


In [52]:
SalePrice

,0
0,100288.900137
1,129530.275043
2,138186.026826
3,156260.445710
4,180220.182826
...,...
1454,73108.420409
1455,78823.053434
1456,126818.379824
1457,101537.331338


In [106]:
submission = pd.concat([id_test2,SalePrice],axis=1)

In [107]:
submission.dtypes

Id      int64
0     float64
dtype: object

In [108]:
submission = submission.rename(columns={0:"SalePrice"})

In [109]:
submission

,Id,SalePrice
0,1461,100288.900137
1,1462,129530.275043
2,1463,138186.026826
3,1464,156260.445710
4,1465,180220.182826
...,...,...
1454,2915,73108.420409
1455,2916,78823.053434
1456,2917,126818.379824
1457,2918,101537.331338


In [110]:
submission.to_csv("submission1.csv")

In [101]:
submission

,Id,SalePrice
0,1461,100288.900137
1,1462,129530.275043
2,1463,138186.026826
3,1464,156260.445710
4,1465,180220.182826
...,...,...
1454,2915,73108.420409
1455,2916,78823.053434
1456,2917,126818.379824
1457,2918,101537.331338


In [82]:
submission.shape

(1459, 2)

In [83]:
submission.reset_index(inplace=True)

In [84]:
submission.drop("index",axis=1,inplace=True)

In [85]:
submission

,Id,SalePrice
0,1461,100288.900137
1,1462,129530.275043
2,1463,138186.026826
3,1464,156260.445710
4,1465,180220.182826
...,...,...
1454,2915,73108.420409
1455,2916,78823.053434
1456,2917,126818.379824
1457,2918,101537.331338
